In [100]:
pip install pyspark

In [101]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, StandardScaler, VectorAssembler, ChiSqSelector
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


In [102]:
# Create a Spark session
spark = SparkSession.builder.appName("BostonHousingFeatureEngineeringDone").getOrCreate()


In [103]:
# 1. Load the dataset into PySpark
# Assume we have a CSV file with headers
data = spark.read.csv("/content/HousingData.csv", header=True, inferSchema=True)

# Print the schema to see data types
data.printSchema()



root
 |-- CRIM: string (nullable = true)
 |-- ZN: string (nullable = true)
 |-- INDUS: string (nullable = true)
 |-- CHAS: string (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: string (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PTRATIO: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: string (nullable = true)
 |-- MEDV: double (nullable = true)



In [104]:
# Define categorical and numerical columns
categorical_cols = ["CHAS", "RAD"]
numerical_cols = ["CRIM", "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "TAX", "PTRATIO", "B", "LSTAT"]

In [105]:
#Import DoubleType
from pyspark.sql.types import DoubleType

In [106]:
# 2. Handle missing values and convert to appropriate types
for col_name in numerical_cols + ["MEDV"]:
    data = data.withColumn(col_name, col(col_name).cast(DoubleType()))

# Use Imputer to handle NaN values in numerical columns
imputer = Imputer(inputCols=numerical_cols, outputCols=numerical_cols)
imputer.setStrategy("mean")  # You can change this to "median" or a constant value


Imputer_867407f8374e

In [107]:
# 3. Handle categorical features
# For categorical columns, replace NaN with a placeholder value before indexing
from pyspark.sql.functions import when, col # Import the when function
for cat_col in categorical_cols:
    data = data.withColumn(cat_col, when(col(cat_col).isNull(), "Unknown").otherwise(col(cat_col)))

indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_indexed", handleInvalid="keep") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=f"{col}_indexed", outputCol=f"{col}_encoded") for col in categorical_cols]

In [108]:
# 4. Scale numerical features
assembler_num = VectorAssembler(inputCols=numerical_cols, outputCol="num_features")
scaler = StandardScaler(inputCol="num_features", outputCol="scaled_num_features", withStd=True, withMean=True)


In [109]:
# 5. Combine features
encoded_cols = [f"{col}_encoded" for col in categorical_cols]
assembler = VectorAssembler(inputCols=encoded_cols + ["scaled_num_features"], outputCol="features")


In [110]:
# 6. Perform feature selection
median_medv = data.approxQuantile("MEDV", [0.5], 0.01)[0]
data = data.withColumn("label", (col("MEDV") > median_medv).cast("double"))

selector = ChiSqSelector(numTopFeatures=10, featuresCol="features", outputCol="selected_features", labelCol="label")



In [111]:
# Create and fit the pipeline
pipeline = Pipeline(stages=[imputer] + indexers + encoders + [assembler_num, scaler, assembler, selector])

try:
    model = pipeline.fit(data)
    result = model.transform(data)

    # Show the result
    result.select("selected_features", "label").show(5, truncate=False)

    # Get feature importances
    feature_importances = model.stages[-1].selectedFeatures
    print("Selected feature indices:", feature_importances)

except Exception as e:
    print(f"An error occurred: {str(e)}")
    # Optionally, you can add more detailed error handling here



+---------------------------------------------------------------------------------------------------------------------------+-----+
|selected_features                                                                                                          |label|
+---------------------------------------------------------------------------------------------------------------------------+-----+
|(10,[5,6,7,8,9],[0.29614984653077786,-1.3097142836688263,-0.14407485473245793,-0.6659491794887344,-1.4575579672895913])    |1.0  |
|(10,[5,6,7,8,9],[-0.4891544449063988,-0.5991779446787058,-0.7395303607434325,-0.9863533804386955,-0.3027944997494501])     |1.0  |
|(10,[5,6,7,8,9],[-0.4891544449063988,-0.5991779446787058,-0.7395303607434325,-0.9863533804386955,-0.3027944997494501])     |1.0  |
|(10,[3,5,6,7,8,9],[1.0,-0.4891544449063988,-1.3291196878849432,-0.8344580501075004,-1.105021603012755,0.11292034856500006])|1.0  |
|(10,[3,5,6,7,8,9],[1.0,-0.4891544449063988,-1.3291196878849432,-0.834458050

In [112]:
# Stop the Spark session
spark.stop()